In [4]:
from utils import progress_bar
import numpy as np
from iotools import loader_factory
import matplotlib.pyplot as plt
%matplotlib inline

def Create_SumN_Rings( DATA_DIRS=['/data/hkml_data/IWCDgrid/varyE/e-'], flavour='100k.h5', 
                      output_file='/data/jamieson/data/e-_tworing.npz', nsum=2 ):
    """
        Build file with nsum events put into a single event.  
        The charges are summed, and the latest time is taken.
        If nsum is a floating point number, then sum a random
        uniform distributed number of electrons (but never give zero!).
        Also, make maximum rings per event 6.0!
    """
    maxrings=6.0
    if type(nsum) == int:
        if nsum > maxrings: nsum = 6
    else:
        if nsum < 1.0 : nsum =1.0
        if nsum > maxrings: nsum = 6.0
        
    loader=loader_factory('H5Dataset', batch_size=1, 
                          shuffle=True, num_workers=4, data_dirs=DATA_DIRS, 
                          flavour=flavour, start_fraction=0.0, use_fraction=0.5,
                          read_keys=[ 'event_data', 'labels','directions', 'energies', 'pids', 'positions'])    
    #output lists
    ev_data=[]
    labels=[]
    pids=[]
    positions=[]
    directions=[]
    energies=[]
    mring_labels=[]
    
    #charge and time data in tqev1,tqev2,tqev3
    tqevs=[]
    tqevsum=[]
    nevavail=0
    nevget=0
    progress = display(progress_bar(0,len(loader)),display_id=True)
    for i,evt in enumerate(loader): 
        if ( i==0 or (i+1)%100==0) :
            progress.update( progress_bar(i+1,len(loader),"Progress"))
            
        while nevget == 0 or nevget > maxrings:
            tqevs = []
            if type(nsum) == int: 
                nevget = nsum
            else:
                nevget = np.random.randint(1,maxrings+1) 
            
        tqevs.append( evt )
        nevavail = nevavail + 1
        #print("nsum,nevavail,nevget=",nsum,nevavail,nevget)
        
        if nevavail == nevget:  # have enough events, now do sum
            for j,tq in enumerate(tqevs): # do sum
                tqev = tq[0][0]
                if j == 0: # initial set
                    tqevsum = np.copy( tqev )
                else:
                    tqevsum[:,:,0] = tqevsum[:,:,0] + tqev[:,:,0]
                    tqevsum[:,:,1] = np.maximum( tqevsum[:,:,1], tqev[:,:,1] )
        
            ev_data.append( np.expand_dims( tqevsum, axis=0 ) )
            labels.append( nevget-1 )
            mring_labels.append( [] )
            directions.append( [] )
            energies.append( [] )  
            pids.append( [] )  
            positions.append( [] )  
            for tq in tqevs:
                mring_labels[-1].append( tq[1] ) 
                directions[-1].append(   tq[2] )
                energies[-1].append(     tq[3] )  
                pids[-1].append(         tq[4] )  
                positions[-1].append(    tq[5] )
            for ifill in range(int(nsum)-len(tqevs)):
                mring_labels[-1].append( tqevs[0][1] ) 
                directions[-1].append(   tqevs[0][2] )
                energies[-1].append(     tqevs[0][3] )  
                pids[-1].append(         tqevs[0][4] )  
                positions[-1].append(    tqevs[0][5] )
                
            nevavail = 0 # reset counters
            nevget = 0
            tqevs=[]
        
    print("Create_SumN_Rings finished loop over events")  
    all_events=np.concatenate(ev_data)
    all_labels=np.asarray(labels)
    all_pids=np.asarray(pids)
    all_positions=np.asarray(positions)
    all_directions=np.asarray(directions)
    all_energies=np.asarray(energies)
    all_mring_labels=np.asarray(mring_labels)
    print("Begin data compress")
    np.savez_compressed(output_file,
                    event_data=all_events,
                    labels=all_labels,
                    pids=all_pids,
                    positions=all_positions,
                    directions=all_directions,
                    energies=all_energies,
                    mring_labels=all_mring_labels
                       )
    print("Done Create_Sum_Rings")


In [5]:
Create_SumN_Rings(  DATA_DIRS=['/data/hkml_data/IWCDgrid/varyE/e-'], flavour='100k.h5', 
                      output_file='/data/jamieson/data/e-_randomto6ring.npz', nsum=6.0 )
print("Done.")

Create_SumN_Rings finished loop over events
Begin data compress
Done Create_Sum_Rings
Done.


In [3]:
Create_SumN_Rings(  DATA_DIRS=['/data/hkml_data/IWCDgrid/varyE/e-'], flavour='100k.h5', 
                      output_file='/data/jamieson/data/e-_tworing.npz', nsum=2 )
print("Done.")

Create_SumN_Rings finished loop over events
Begin data compress



KeyboardInterrupt



In [ ]:
Create_SumN_Rings(  DATA_DIRS=['/data/hkml_data/IWCDgrid/varyE/e-'], flavour='100k.h5', 
                      output_file='/data/jamieson/data/e-_threering.npz', nsum=3 )
print("Done.")